In [106]:
import numpy as np

size = 1000
np.random.seed(123)
x = np.random.uniform(size=size)

np.random.seed(456)
y = 3*x+ np.random.uniform(size=size)

np.random.seed(789)
z =  5*y+ np.random.uniform(size=size)

np.random.seed(135)
w = 2*x + 7*z+np.random.uniform(size=size)

#xs = np.array([x,y,z,w]).T
xs = np.array([y,z,w,x]).T

In [107]:
xs

array([[  2.33816347e+00,   1.20141522e+01,   8.61533324e+01,
          6.96469186e-01],
       [  1.02148479e+00,   5.33767105e+00,   3.82636707e+01,
          2.86139335e-01],
       [  1.46419762e+00,   8.11476695e+00,   5.74524697e+01,
          2.26851454e-01],
       ..., 
       [  1.66909717e+00,   9.18710623e+00,   6.52045372e+01,
          3.47146060e-01],
       [  3.86523572e-01,   2.52848875e+00,   1.79734284e+01,
          4.16848861e-03],
       [  9.23529229e-01,   4.81468494e+00,   3.43320338e+01,
          2.94894709e-01]])

In [108]:
from sklearn.decomposition import FastICA

n_samples, n_features = xs.shape
ica = FastICA(random_state=1234, max_iter=1000).fit(xs)
W_ica = np.linalg.pinv(ica.mixing_)
print(W_ica)

[[  5.58006057e-01  -1.15809196e-01   6.66050696e-04  -7.39249524e-03]
 [  2.44834816e-03   1.16880693e-02  -1.69841457e-03   1.10164061e-01]
 [  1.06093244e-02   7.51532929e-01  -1.07695259e-01   2.22932530e-01]
 [  1.17882991e-01   1.45524643e-03  -4.76533579e-04  -3.22730159e-01]]


In [109]:
W_ica_ = 1 / np.abs(W_ica) # もちろんこの前に成分が0でないことはチェックする
print(W_ica_)

[[  1.79209524e+00   8.63489288e+00   1.50138722e+03   1.35272322e+02]
 [  4.08438642e+02   8.55573297e+01   5.88784399e+02   9.07737053e+00]
 [  9.42567091e+01   1.33061369e+00   9.28545980e+00   4.48566209e+00]
 [  8.48298800e+00   6.87168841e+02   2.09848801e+03   3.09856384e+00]]


In [110]:
from copy import deepcopy
from munkres import Munkres
m = Munkres()
ixs = np.vstack(m.compute(deepcopy(W_ica_)))
print(ixs)

[[0 1]
 [1 3]
 [2 2]
 [3 0]]


In [111]:
ixs = ixs[np.argsort(ixs[:, 0]), :]
ixs_perm = ixs[:, 1] # ixs_perm = [2 0 3 1]
W_ica_perm = np.zeros_like(W_ica)
W_ica_perm[ixs_perm] = W_ica
print(W_ica_perm)

[[  1.17882991e-01   1.45524643e-03  -4.76533579e-04  -3.22730159e-01]
 [  5.58006057e-01  -1.15809196e-01   6.66050696e-04  -7.39249524e-03]
 [  1.06093244e-02   7.51532929e-01  -1.07695259e-01   2.22932530e-01]
 [  2.44834816e-03   1.16880693e-02  -1.69841457e-03   1.10164061e-01]]


In [112]:
D = np.diag(W_ica_perm)[:, np.newaxis]
W_ica_perm_D = W_ica_perm / D
print(W_ica_perm_D)

[[  1.00000000e+00   1.23448380e-02  -4.04242863e-03  -2.73771607e+00]
 [ -4.81832253e+00   1.00000000e+00  -5.75127641e-03   6.38334045e-02]
 [ -9.85124549e-02  -6.97832880e+00   1.00000000e+00  -2.07003105e+00]
 [  2.22245635e-02   1.06096936e-01  -1.54171383e-02   1.00000000e+00]]


In [113]:
b_est = np.eye(n_features) - W_ica_perm_D
print(b_est)

[[  0.00000000e+00  -1.23448380e-02   4.04242863e-03   2.73771607e+00]
 [  4.81832253e+00   0.00000000e+00   5.75127641e-03  -6.38334045e-02]
 [  9.85124549e-02   6.97832880e+00   0.00000000e+00   2.07003105e+00]
 [ -2.22245635e-02  -1.06096936e-01   1.54171383e-02   0.00000000e+00]]


In [114]:
def _slttestperm(b_i):
# b_iの行を並び替えて下三角行列にできるかどうかチェック
    n = b_i.shape[0]
    remnodes = np.arange(n)
    b_rem = deepcopy(b_i)
    p = list() 

    for i in range(n):
        # 成分が全て0である行番号のリスト
        ixs = np.where(np.sum(np.abs(b_rem), axis=1) < 1e-12)[0]

        if len(ixs) == 0:
            return None
        else:
            ix = ixs[0]
            p.append(remnodes[ix])

            #　成分が全て0である行を削除
            remnodes = np.hstack((remnodes[:ix], remnodes[(ix + 1):]))
            ixs = np.hstack((np.arange(ix), np.arange(ix + 1, len(b_rem))))
            b_rem = b_rem[ixs, :]
            b_rem = b_rem[:, ixs]

    return np.array(p)

b = b_est
n = b.shape[0]
assert(b.shape == (n, n))

ixs = np.argsort(np.abs(b).ravel())
#print(np.abs(b).ravel())
print(ixs)
#print(range(int(n * (n + 1) / 2) - 1, (n * n) - 1))

for i in range(int(n * (n + 1) / 2) - 1, (n * n) - 1):
    b_i = deepcopy(b)
    b_i.ravel()[ixs[:i]] = 0
    print(b_i)
    ixs_perm = _slttestperm(b_i)
    print(ixs_perm)
    if ixs_perm is not None:
        print(ixs_perm)
        b_opt = deepcopy(b)
        #print(b_opt)
        b_opt = b_opt[ixs_perm, :]
        #print(b_opt)
        b_opt = b_opt[:, ixs_perm]
        #print(b_opt)
        break

#print(b_opt)
b_csl = np.tril(b_opt, -1)
#print(b_csl)
#print(ixs_perm)
b_csl[ixs_perm, :] = deepcopy(b_csl)
#print(b_csl)
b_csl[:, ixs_perm] = deepcopy(b_csl)
print(b_csl)


[ 0  5 10 15  2  6  1 14 12  7  8 13 11  3  4  9]
[[ 0.          0.          0.          2.73771607]
 [ 4.81832253  0.          0.         -0.0638334 ]
 [ 0.09851245  6.9783288   0.          2.07003105]
 [ 0.         -0.10609694  0.          0.        ]]
None
[[ 0.          0.          0.          2.73771607]
 [ 4.81832253  0.          0.          0.        ]
 [ 0.09851245  6.9783288   0.          2.07003105]
 [ 0.         -0.10609694  0.          0.        ]]
None
[[ 0.          0.          0.          2.73771607]
 [ 4.81832253  0.          0.          0.        ]
 [ 0.          6.9783288   0.          2.07003105]
 [ 0.         -0.10609694  0.          0.        ]]
None
[[ 0.          0.          0.          2.73771607]
 [ 4.81832253  0.          0.          0.        ]
 [ 0.          6.9783288   0.          2.07003105]
 [ 0.          0.          0.          0.        ]]
[3 0 1 2]
[3 0 1 2]
[[ 0.          0.          0.          2.73771607]
 [ 4.81832253  0.          0.         -0.063